# Questão 1: Cálculo de Lucro com SELIC

## Passo 1 - Instalar e importar módulos e bibliotecas

In [1]:
from datetime import datetime, date
from matplotlib import pyplot as plt 
import numpy as np
from bcb import sgs

## Passo 2 - Coletar dados do usuário

In [2]:
capital = float(input("Digite o capital investido: "))
frequencia = input("Digite a frequência do período (Y, M, D): ")
inicio = input("Digite a data inicial madior doque 1995/01/01 no formato YYYY/MM/DD: ")
fim = input("Digite a data final no seguinte formato YYYY/MM/DD: ")

Digite o capital investido: 1000
Digite a frequência do período (Y, M, D): M
Digite a data inicial madior doque 1995/01/01 no formato YYYY/MM/DD: 1996/01/01
Digite a data final no seguinte formato YYYY/MM/DD: 2023/12/31


In [3]:
if frequencia == 'M':
    frequencia = 'ME'
frequencia

'ME'

## Passo 3 - Tratar dados coletados

In [4]:
data_inicio = datetime.strptime(inicio,"%Y/%m/%d").date()
data_fim = datetime.strptime(fim,"%Y/%m/%d").date()

data_inicio

datetime.date(1996, 1, 1)

## Passo 4 - Pegar dados do CDI do banco central

In [6]:
taxas_selic = sgs.get({"selic": 11}, start = data_inicio, end = data_fim)

taxas_selic

,selic
Date,
1996-01-02,0.120000
1996-01-03,0.119667
1996-01-04,0.114667
1996-01-05,0.113667
1996-01-08,0.113667
...,...
2023-12-22,0.043739
2023-12-26,0.043739
2023-12-27,0.043739


In [7]:
taxas_selic = taxas_selic/100
taxas_selic

,selic
Date,
1996-01-02,0.001200
1996-01-03,0.001197
1996-01-04,0.001147
1996-01-05,0.001137
1996-01-08,0.001137
...,...
2023-12-22,0.000437
2023-12-26,0.000437
2023-12-27,0.000437


## Passo 5 - Calcular o retorno do período

In [8]:
capital_acumulado = capital * (1+taxas_selic["selic"]).cumprod()-1
capital_acumulado

Date
1996-01-02     1000.200000
1996-01-03     1001.398106
1996-01-04     1002.547526
1996-01-05     1003.688228
1996-01-08     1004.830227
                  ...     
2023-12-22    39888.711891
2023-12-26    39906.159252
2023-12-27    39923.614244
2023-12-28    39941.076872
2023-12-29    39958.547137
Name: selic, Length: 7029, dtype: float64

In [9]:
capital_com_frequencia = capital_acumulado.resample(frequencia).last()
capital_com_frequencia

Date
1996-01-31     1024.764965
1996-02-29     1048.885472
1996-03-31     1072.201557
1996-04-30     1094.395862
1996-05-31     1116.443476
                  ...     
2023-08-31    38482.736539
2023-09-30    38857.145487
2023-10-31    39244.781610
2023-11-30    39604.268192
2023-12-31    39958.547137
Freq: ME, Name: selic, Length: 336, dtype: float64

# Questão 2: Período Mais Lucrativo de 500 Dias

## Passo1 - Filtrar os dados da SELIC para o período de interesse (2000-01-01 a 2022-03-31).

In [12]:
data_inicio_2 = date(2000,1,1)
data_fim_2 = date(2022,3,31)

In [13]:
taxas_selic_2 = sgs.get({"selic": 11}, start = data_inicio_2, end = data_fim_2)/100

taxas_selic_2

,selic
Date,
2000-01-03,0.000692
2000-01-04,0.000692
2000-01-05,0.000692
2000-01-06,0.000693
2000-01-07,0.000693
...,...
2022-03-25,0.000437
2022-03-28,0.000437
2022-03-29,0.000437


## Passo 2 - Calcular a rentabilidade em janelas de 500 dias.

In [17]:
janelas_500_dias = ((1 + taxas_selic_2).rolling(window = 500).apply(np.prod) -1)
janelas_500_dias

,selic
Date,
2000-01-03,NaN
2000-01-04,NaN
2000-01-05,NaN
2000-01-06,NaN
2000-01-07,NaN
...,...
2022-03-25,0.086543
2022-03-28,0.086864
2022-03-29,0.087185


In [18]:
#transformar o index (date) em uma coluna
janelas_500_dias = janelas_500_dias.reset_index()
janelas_500_dias

,Date,selic
0,2000-01-03,NaN
1,2000-01-04,NaN
2,2000-01-05,NaN
3,2000-01-06,NaN
4,2000-01-07,NaN
...,...,...
5583,2022-03-25,0.086543
5584,2022-03-28,0.086864
5585,2022-03-29,0.087185
5586,2022-03-30,0.087506


## Passo 3 - Gerar o intervalo de datas dentro da tabela de cálculo.

In [20]:
janelas_500_dias["data_inicio"] = janelas_500_dias["Date"].shift(500)
janelas_500_dias

,Date,selic,data_inicio
0,2000-01-03,NaN,NaT
1,2000-01-04,NaN,NaT
2,2000-01-05,NaN,NaT
3,2000-01-06,NaN,NaT
4,2000-01-07,NaN,NaT
...,...,...,...
5583,2022-03-25,0.086543,2020-03-27
5584,2022-03-28,0.086864,2020-03-30
5585,2022-03-29,0.087185,2020-03-31
5586,2022-03-30,0.087506,2020-04-01


In [21]:
janelas_500_dias = janelas_500_dias.dropna()
janelas_500_dias

,Date,selic,data_inicio
500,2002-01-02,0.377688,2000-01-03
501,2002-01-03,0.377689,2000-01-04
502,2002-01-04,0.377689,2000-01-05
503,2002-01-07,0.377688,2000-01-06
504,2002-01-08,0.377687,2000-01-07
...,...,...,...
5583,2022-03-25,0.086543,2020-03-27
5584,2022-03-28,0.086864,2020-03-30
5585,2022-03-29,0.087185,2020-03-31
5586,2022-03-30,0.087506,2020-04-01


In [22]:
janelas_500_dias.columns = ["data_final","retorno_selic_500d","data_inicial"]
janelas_500_dias

,data_final,retorno_selic_500d,data_inicial
500,2002-01-02,0.377688,2000-01-03
501,2002-01-03,0.377689,2000-01-04
502,2002-01-04,0.377689,2000-01-05
503,2002-01-07,0.377688,2000-01-06
504,2002-01-08,0.377687,2000-01-07
...,...,...,...
5583,2022-03-25,0.086543,2020-03-27
5584,2022-03-28,0.086864,2020-03-30
5585,2022-03-29,0.087185,2020-03-31
5586,2022-03-30,0.087506,2020-04-01


## Passo 4 - Identificar o maior retorno dentro da tabela de períodos.

In [23]:
maior_retorno = janelas_500_dias["retorno_selic_500d"].max()
maior_retorno

0.4669665013674005

In [27]:
gabarito = janelas_500_dias[janelas_500_dias["retorno_selic_500d"]==maior_retorno]
gabarito

,data_final,retorno_selic_500d,data_inicial
956,2003-10-22,0.466967,2001-10-26
